In [52]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from scripts.load_data import load_postings, load_votes, get_first_contact_df, subset_users
from tqdm import tqdm

In [53]:
votes = load_votes("input/")
postings = load_postings("input/")

------
Ana doing things that got done

In [109]:
# user_categories = {}
# for user in postings_sub['ID_CommunityIdentity'].unique():
    
#     user_categories[user] = set(postings_sub.query("ID_CommunityIdentity == @user")['ArticleRessortName'].values)

In [ ]:
# def jaqquard_similarity(a, b):
#     return len(a & b) / len(a | b)

In [1]:

# import warnings
# warnings.filterwarnings("ignore")

# groups = postings_sub.groupby("ID_CommunityIdentity")

# all_groups = groups.groups.keys()
# groups_list = list(all_groups)
# list_len = len(groups_list)

# df = pd.DataFrame(columns=('UserA', 'UserB', 'topics_num'))

# for i in range (list_len):
#     for j in range(i+1, list_len):
#         id_a = groups_list[i]
#         id_b = groups_list[j]
#         group_a = groups.get_group(id_a)
#         group_b = groups.get_group(id_b)
#         topics_num = len(np.intersect1d(group_a.ArticleRessortName, group_b.ArticleRessortName))

#         print(str(id_a)+ ", "+ str(id_b)+ ": "+ str(topics_num))

#         df.append({'UserA': id_a, 'UserB': id_b, 'topics_num': topics_num}, ignore_index=True)

# df


In [ ]:
# user_resort = {}
# user_channel = {}
# for user in postings['ID_CommunityIdentity'].unique():
    
#     user_resort[user] = set(postings.query("ID_CommunityIdentity == @user")['ArticleRessortName'].values)
#     user_channel[user] = set(postings.query("ID_CommunityIdentity == @user")['ArticleChannel'].values)
# arr = []
# users = list(user_channel.keys())
# alfa = 0.3

# for user_a in tqdm(users):
#     user_channel_a = user_channel[user_a]
#     user_resort_a = user_resort[user_a]
    
#     for user_b in users:
#         if user_a != user_b:
#             user_channel_b = user_channel[user_b]
#             user_resort_b = user_resort[user_b]

#             similarity_channel = jaqquard_similarity(user_channel_a, user_channel_b)
#             similarity_resort = jaqquard_similarity(user_resort_a, user_resort_b)
#             weighted_similarity = alfa * similarity_channel + (1 - alfa) * similarity_resort

#             arr.append([user_a, user_b, weighted_similarity])

# arr

# pd.DataFrame(arr, columns=["A", "B", "Similarity"]).set_index(["A", "B"])

-----------------
playing with weighted similarity for channel and resort

In [ ]:
#####################################################################################
######the code below completly taken from notebook.ipynb ##################################

import itertools
# one can adapt this with e.g. ressort instead of article

def create_graph(df, article_or_ressort = "ID_Posting", user="UserVote"):
    graph = nx.Graph()
    graph.add_nodes_from(df[article_or_ressort].unique())
    graph.add_nodes_from(df[user].unique())
    graph.add_edges_from(list(map(tuple, df[[article_or_ressort, user]].drop_duplicates().values)))
    graph = graph.to_undirected()
    return graph

def compute_overlap(graph, df, article_or_ressort,verbose=False):
    uu_overlap = {}
    article_ids = df[article_or_ressort].unique()
    for idx, article in enumerate(article_ids):
        if verbose: print(round((idx/len(article_ids))*100), "%", end="\r")
        users_commented =list(graph.neighbors(article))
        for uu_tuple in itertools.product(users_commented, users_commented):
            if uu_tuple[0] != uu_tuple[1]:
                if uu_tuple[0] > uu_tuple[1]:
                    uu_tuple = (uu_tuple[1], uu_tuple[0])
                if uu_tuple in uu_overlap:
                    uu_overlap[uu_tuple] += 1
                else:
                    uu_overlap[uu_tuple] = 1
                    
    return uu_overlap


def user_lookup_df(df, article_or_ressort="ID_Posting"):
    user_num_articles = df[["UserVote", article_or_ressort]].drop_duplicates()\
        .groupby(["UserVote"]).size().to_frame()
    # make dict of users and the number of articles they voted on
    user_num_articles = dict(zip(user_num_articles.index, user_num_articles[0]))
    return user_num_articles

def compute_similarity(uu_overlap, user_num_articles, chunckIdx):
    similarities = []
    for uu_tuple in uu_overlap.keys():
        overlap = uu_overlap[uu_tuple]
        try:
            union = user_num_articles[uu_tuple[0]] + user_num_articles[uu_tuple[1]] 
        except:
            print(uu_tuple)
        similarities += [[uu_tuple[0],uu_tuple[1], overlap/union ]]
    return pd.DataFrame(similarities, columns=["A", "B", f"Similarity_{chunckIdx}"]).set_index(["A", "B"])

def compute_time_base_similiarities(selected_postings, article_or_ressort, num_chunks=30):
    sum_sims= 0
    n = 0
    chunks = []
    running_mean = []
    for chunckIdx, subset_df  in enumerate(np.array_split(selected_postings,num_chunks)):
        print(round(chunckIdx/num_chunks) *100, " %", end="\r")
        graph_ressort = create_graph(subset_df, article_or_ressort, "UserCommunityName")
        uu_overlap_ressort = compute_overlap(graph_ressort, subset_df, article_or_ressort)
        user_num_article_or_ressort = user_lookup_df(subset_df, article_or_ressort)
        similarity_table_ressort = compute_similarity(uu_overlap_ressort, user_num_article_or_ressort,chunckIdx)
        n += similarity_table_ressort.shape[0]
        sum_sims += similarity_table_ressort.sum().item()
        chunks += [similarity_table_ressort.mean()]
        running_mean += [sum_sims / n]
    return chunks, running_mean

In [ ]:
user_selection = subset_users(votes, postings, "both", num_days_min=30, firt_interaction_middle=True)

selected_postings = postings[postings["UserCommunityName"].isin(user_selection)].sort_values("PostingCreatedAt")
selected_postings["UserCommunityName"] = "user_" + selected_postings["UserCommunityName"]

In [ ]:
### Similarity based on article topic
time_similarity_channel, time_similarity_running_channel = compute_time_base_similiarities(selected_postings, "ArticleRessortName")
time_similarity_ressort, time_similarity_running_ressort = compute_time_base_similiarities(selected_postings, "ArticleChannel")

postings['ArticleRessortName'].nunique()
postings['ArticleChannel'].nunique()
alfa = (postings['ArticleChannel'].nunique())/(postings['ArticleChannel'].nunique()+postings['ArticleRessortName'].nunique())
print(alfa)
# alfa = 0.3

time_similarity_channel_alfa = [x * alfa for x in time_similarity_channel]
time_similarity_ressort_alfa = [x * (1-alfa) for x in time_similarity_ressort]

time_similarity_running_channel_alfa = [x * alfa for x in time_similarity_running_channel]
time_similarity_running_ressort_alfa = [x * (1-alfa) for x in time_similarity_running_ressort]


time_similarity_final = [x + y for x, y in zip([x * alfa for x in time_similarity_channel], [x * (1-alfa) for x in time_similarity_ressort])]
time_similarity_running_final = [x + y for x, y in zip([x * alfa for x in time_similarity_running_channel], [x * (1-alfa) for x in time_similarity_running_ressort])]

plt.plot(range(1,31), time_similarity_final)
plt.plot(range(1,31), time_similarity_running_final)
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for alfa in np.arange(0, 1.1, 0.01):
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + str(alfa),
            x=np.arange(1, 31, 1),
            y = [x + y for x, y in zip([x * alfa for x in time_similarity_running_channel], [x * (1-alfa) for x in time_similarity_running_ressort])]

))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in np.arange(0, 1.01, 0.01):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to alfa: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][int(i/0.01)] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0.5,
    currentvalue={"prefix": "Alfa: "},
    steps=steps,
    
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

-------------------

## Votes similarity
adjusted functions from notebook.ipynb

In [ ]:
merged = pd.merge(votes, postings, on="ID_Posting")
merged["Vote"] = merged["VotePositive"] - merged["VoteNegative"]
merged = merged.rename(columns={"UserCommunityName_x": "UserVote", "UserCommunityName_y": "UserPost"})
merged = merged[merged["UserVote"].isin(user_selection)].sort_values("VoteCreatedAt")
merged = merged[["ID_Posting", "UserPost", "UserVote", "Vote", "VoteCreatedAt"]]
merged.head()

In [ ]:
def compute_overlap_votes(graph_pos, graph_neg, df, article_or_ressort="ID_Posting",verbose=False):
    uu_overlap = {}
    article_ids = df[article_or_ressort].unique()
    for idx, article in enumerate(article_ids):

        try:
            users_voted_pos =list(graph_pos.neighbors(article))
            for uu_tuple in itertools.product(users_voted_pos, users_voted_pos):
                if uu_tuple[0] != uu_tuple[1]:
                    if uu_tuple[0] > uu_tuple[1]:
                        uu_tuple = (uu_tuple[1], uu_tuple[0])
                    if uu_tuple in uu_overlap:
                        uu_overlap[uu_tuple] += 1
                    else:
                        uu_overlap[uu_tuple] = 1
        except:
            users_voted_pos = []

        try:
            users_voted_neg =list(graph_neg.neighbors(article))
            for uu_tuple in itertools.product(users_voted_neg, users_voted_neg):
                if uu_tuple[0] != uu_tuple[1]:
                    if uu_tuple[0] > uu_tuple[1]:
                        uu_tuple = (uu_tuple[1], uu_tuple[0])
                    if uu_tuple in uu_overlap:
                        uu_overlap[uu_tuple] += 1
                    else:
                        uu_overlap[uu_tuple] = 1
        except:
            users_voted_neg = []

        if (len(users_voted_neg) != 0 & len(users_voted_pos) != 0):
            for uu_tuple in itertools.product(users_voted_pos, users_voted_neg):
                if uu_tuple[0] != uu_tuple[1]:
                    if uu_tuple[0] > uu_tuple[1]:
                        uu_tuple = (uu_tuple[1], uu_tuple[0])
                    if uu_tuple in uu_overlap:
                        uu_overlap[uu_tuple] -= 1
                    else:
                        uu_overlap[uu_tuple] = -1
                    
            
    return uu_overlap


def compute_time_base_similiarities_votes(merged, article_or_ressort="ID_Posting", num_chunks=30):
    sum_sims= 0
    n = 0
    chunks = []
    running_mean = []
    for chunckIdx, subset_df  in tqdm(enumerate(np.array_split(merged,num_chunks))):
        print(round(chunckIdx/num_chunks) *100, " %", end="\r")
        positive = subset_df[subset_df["Vote"] == 1]
        negative = subset_df[subset_df["Vote"] == -1]
        graph_pos = create_graph(positive, article_or_ressort, "UserVote")
        graph_neg = create_graph(negative, article_or_ressort, "UserVote")
        uu_overlap_ressort = compute_overlap_votes(graph_pos, graph_neg, subset_df, article_or_ressort)
        user_num_article_or_ressort = user_lookup_df(subset_df, article_or_ressort)
        similarity_table_ressort = compute_similarity(uu_overlap_ressort, user_num_article_or_ressort,chunckIdx)
        n += similarity_table_ressort.shape[0]
        sum_sims += similarity_table_ressort.sum().item()
        chunks += [similarity_table_ressort.mean()]
        running_mean += [sum_sims / n]
    return chunks, running_mean


In [ ]:
time_similarity, time_similarity_running = compute_time_base_similiarities_votes(merged, "ID_Posting", 30)

In [ ]:
fig, ax = plt.subplots()

ax.annotate(f'First Contact (replies or votes)', xy=(15, 0.5), xytext=(1, 0.52),
                    arrowprops=dict(facecolor='black', shrink=0.05, width=2, headwidth=7))

plt.title(f'Similarity over time (Votes)')
plt.xlabel("days")
plt.ylabel("Similarity")
plt.plot(range(1, 31), time_similarity, alpha=0.1, color="blue")
plt.plot(range(1, 31), time_similarity_running, color="green")
plt.legend(["Actual values", "Running mean"])
plt.axvline(x=15, color="green", linestyle="--")
plt.show()
plt.clf()